# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_file = "output/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

base_url = "http://api.openweathermap.org/data/2.5/weather?"

units = 'Imperial'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
city_lat = []
city_lng = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        city_lat.append(lat_lng[0])
        city_lng.append(lat_lng[1])

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
temp = []
humid = []
clouds = []
wind = []

for city in cities:
    target_city = city
    target_url = f'{base_url}q={target_city}&appid={weather_api_key}&units={units}'
    target_city_index = cities.index(city) + 1
    city_data = requests.get(target_url).json()
    
    print(f'#{target_city_index} - Getting data from {target_city}')
    
    try:
        temp.append(city_data['main']['temp'])
    except:
        temp.append("NAN")
        print(f'No temp data for {target_city}, skipping')
        
    try:
        humid.append(city_data['main']['humidity'])
    except:
        humid.append("NAN")
        print(f'No humid data for {target_city}, skipping')
        
    try:
        clouds.append(city_data['clouds']['all'])
    except:
        clouds.append("NAN")
        print(f'No cloud data for {target_city}, skipping')

    try:
        wind.append(city_data['wind']['speed'])
    except:
        wind.append("NAN")
        print(f'No wind data for {target_city}, skipping')
        
print(len(temp))
print(len(humid))
print(len(clouds))
print(len(wind))

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_df = pd.DataFrame({'City': cities, 'Latitude': city_lat, 'Longitude': city_lng, 'Temperature \N{DEGREE SIGN}F': temp, 'Humidity (%)': humid, 'Cloud Coverage (%)': clouds, 'Wind Speed (MPH)': wind})
weather_df['City'] = weather_df['City'].str.title()

weather_df['Temperature \N{DEGREE SIGN}F'] = weather_df['Temperature \N{DEGREE SIGN}F'].astype(float)
weather_df['Humidity (%)'] = weather_df['Humidity (%)'].astype(float)
weather_df['Cloud Coverage (%)'] = weather_df['Cloud Coverage (%)'].astype(float)
weather_df['Wind Speed (MPH)'] = weather_df['Wind Speed (MPH)'].astype(float)

# to csv
weather_df.to_csv(output_file)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humid_over_100 = weather_df.loc[weather_df['Humidity (%)'] >= 100]
print(f'Number of cities where humidity > 100% = {len(humid_over_100)}')
print(f'Removing these {len(humid_over_100)} cities')
weather_df = weather_df.loc[weather_df['Humidity (%)'] < 100]
print(f'Now there is {len(weather_df)} cities in the dataframe.')

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(weather_df['Latitude'], weather_df['Temperature \N{DEGREE SIGN}F'])
plt.title('Temp vs Lat of Various Cities')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.savefig('output/Temp-vs-Lat.png')
print(f"This plot shows that temperature rises as you get closer to the equator.")

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(weather_df['Latitude'], weather_df['Humidity (%)'])
plt.title('Humidity vs Lat of Various Cities')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.savefig('output/Humidity-vs-Lat.png')
print(f"Doesn't appear to be a clear relationship between humidity and latitude.")

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(weather_df['Latitude'], weather_df['Cloud Coverage (%)'])
plt.title('Cloudiness vs Lat of Various Cities')
plt.xlabel('Latitude')
plt.ylabel('Cloud Coverage (%)')
plt.savefig('output/Cloudiness-vs-Lat.png')
print(f"This API source seems to measure cloudiness in multiples of 5. There are clear lines at certain levels of cloud coverage. Because of this, it's hard to see a relationship between cloudiness and latitude.")

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(weather_df['Latitude'], weather_df['Wind Speed (MPH)'])
plt.title('Wind Speed vs Lat of Various Cities')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (MPH)')
plt.savefig('output/Wind-Speed-vs-Lat.png')
print(f"There doesn't appear to be a relationship between wind speed and latitude. There must be a different factor that relates to wind speed, perhaps longitude, for example.")

## Linear Regression

In [ ]:
north = weather_df.loc[weather_df['Latitude'] >= 0]
south = weather_df.loc[weather_df['Latitude'] <= 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = north['Latitude']
y_values = north['Temperature \N{DEGREE SIGN}F']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(0,-15),fontsize=15,color="red")
plt.title('Temperature vs Lat: North')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.savefig('output/Temp-vs-Lat-North.png', format='png')
print('Negative correlation between temperature and latitude in the northern hemisphere.')

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = south['Latitude']
y_values = south['Temperature \N{DEGREE SIGN}F']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-80,20),fontsize=15,color="red")
plt.title('Temperature vs Lat: Southern Hemisphere')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.savefig('output/Temp-vs-Lat-South.png')
print('Positive correlation between temperature and latitude in the southern hemisphere.')

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = north['Latitude']
y_values = north['Humidity (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(40,10),fontsize=15,color="red")
plt.title('Humidity vs Lat: Northern Hemisphere')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.savefig('output/Humidity-vs-Lat-North.png')
print('Positive correlation between humidity and latitude in the northern hemisphere. Though, it is a pretty weak correlation.')

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = south['Latitude']
y_values = south['Humidity (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-90,15),fontsize=15,color="red")
plt.title('Humidity vs Lat: South')
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.savefig('output/Humidity-vs-Lat-South.png')
print('Positive correlation between humidity and latitude in the southern hemisphere. Like the northern hemisphere, this is also a weak correlation.')

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = north['Latitude']
y_values = north['Cloud Coverage (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(40,65),fontsize=15,color="red")
plt.title('Cloudiness vs Lat: North')
plt.xlabel('Latitude')
plt.ylabel('Cloud Coverage (%)')
plt.savefig('output/Cloudiness-vs-Lat-North.png')
print('No correlation between cloudiness and latitude in the northern hemisphere, that we can see. While the regression line has a positive slope, the distribution of data points does not visually mirror the trendline.')

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = south['Latitude']
y_values = south['Cloud Coverage (%)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-80,60),fontsize=15,color="red")
plt.title('Cloudiness vs Lat: South')
plt.xlabel('Latitude')
plt.ylabel('Cloud Coverage (%)')
plt.savefig('output/Cloudiness-vs-Lat-South.png')
print('No correlation between cloudiness and latitude in the southern hemisphere, that we can see. While the regression line has a positive slope, the distribution of data points does not visually mirror the trendline.')

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = north['Latitude']
y_values = north['Wind Speed (MPH)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(0,35),fontsize=15,color="red")
plt.title('Wind Speed vs Lat: North')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (MPH)')
plt.savefig('output/Wind-Speed-vs-Lat-North.png')
print('No correlation between wind speed and latitude.')

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = south['Latitude']
y_values = south['Wind Speed (MPH)']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-87,28),fontsize=15,color="red")
plt.title('Wind Speed vs Lat: South')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (MPH)')
plt.savefig('output/Wind-Speed-vs-Lat-South.png')
print('No correlation between wind speed and latitude.')